In [1]:
import warnings
from silence_tensorflow import silence_tensorflow
warnings.filterwarnings('ignore')
silence_tensorflow() 

In [2]:
from nn_meter.builder.backends import connect_backend
from nn_meter.builder import create_testcases, run_testcases, detect_fusionrule

The first step to run rule_tester is to prepare a backends and create a workspace for experiment. Users could create a workspace folder by running the following command:

``` Bash
# for TFLite platform
nn-meter create --tflite-workspace <path/to/place/workspace/>

# for OpenVINO platform
nn-meter create --openvino-workspace <path/to/place/workspace/>

# for customized platform
nn-meter create --customized-workspace <backend-name> <path/to/place/workspace/>
```

After running the command, a workspace folder will be created and two yaml files named `backend_config.yaml` and `ruletest_config.yaml`, i.e., configuration file for backend and rule_tester respectively, will be placed in `<workspace-path>/configs/`. Users could edit the content to change configuration. The config will take effect after the the config file is saved and closed.

After creating the workspace and completing configuration, users could initialize workspace in `builder_config` module for experiments:

In [4]:
# initialize workspace in code
workspace_path = "./tftest" # text the path to the workspace folder. refer to ./backend.md for further information.
from nn_meter.builder.utils import builder_config
builder_config.init("tflite", workspace_path)

After creating the workspace, a yaml file named `ruletest_config.yaml` will be placed in `<workspace-path>/configs/`. Users could open `<workspace-path>/configs/ruletest_config.yaml` and edit the content. The config will take effect after the the config file is saved and closed.

Following configuration from `<workspace-path>/configs/ruletest_config.yaml`, the test cases can be created by running:

In [9]:
# generate testcases
origin_testcases = create_testcases()

The test case models will be saved in `<workspace-path>/testcases_ruletest/`, and the test case dictionary will be saved in `<workspace-path>/results/origin_testcases.json`. 

If the test cases has been created before, users could directly use the generated json file of test cases to avoid creating test cases again:

In [ ]:
import os
origin_testcases = os.path.join(workspace_path, "results", "origin_testcases.json")

Next step is to run and profile test cases on the backend. Given required backend, users could run test cases model and get profiled latency value by running:

In [7]:
# connect to backend
backend = connect_backend(backend='tflite_cpu')

# run testcases and collect profiling results
profiled_testcases = run_testcases(backend, origin_testcases)


`backend` refers to the framework of the platform and device to execute the model. Currently we provide three instance on two platforms, i.e., CPU backend, GPU backend with TFLite platform, and VPU backend with OpenVINO platform. Refer to [backend guidance](../docs/builder/backend.md) for how to setup the device and get connection to the backend. In case having run test cases before, user could also use test case json file to avoid running test cases again:

In [9]:
profiled_testcases = os.path.join(workspace_path, "results", "profiled_testcases.json")

Finally, users could detect the fusion rule according to profiled test cases by running the following code. After running `detect_fusionrule`, a json file named `<workspace-path>/results/detected_testcases.json` will be created as the detection result. The result shows each test case obeys the fusion rule or not.

In [8]:
# determine fusion rules from profiling results
detected_testcases = detect_fusionrule(profiled_testcases)